https://github.com/dredwardhyde/gpt-neo-fine-tuning-example/blob/main/gpt_neo.py

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

In [ ]:
!wget https://github.com/dredwardhyde/gpt-neo-fine-tuning-example/raw/main/netflix_titles.csv

In [ ]:
! wget https://github.com/tloen/alpaca-lora/raw/main/alpaca_data.json

--2023-07-19 15:17:18--  https://github.com/tloen/alpaca-lora/raw/main/alpaca_data.json
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tloen/alpaca-lora/main/alpaca_data.json [following]
--2023-07-19 15:17:19--  https://raw.githubusercontent.com/tloen/alpaca-lora/main/alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22773992 (22M) [text/plain]
Saving to: ‘alpaca_data.json’

alpaca_data.json    100%[===================>]  21.72M  --.-KB/s    in 0.1s    

2023-07-19 15:17:20 (184 MB/s) - ‘alpaca_data.json’ saved [22773992/22773992]



In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.manual_seed(42)
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m", bos_token='<|startoftext|>',
#                                           eos_token='<|endoftext|>', pad_token='<|pad|>')
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
tokenizer.pad_token = tokenizer.eos_token
## don't change bos_token, eos_token and pad_token.
## set pad_token as eos_token as done in many tutorials

model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m").cuda()

model.resize_token_embeddings(len(tokenizer))

# descriptions = pd.read_csv('/content/netflix_titles.csv')['description']

max_length = max([len(tokenizer.encode(description)) for description in descriptions])
# alpacadata['train']['prompt']

print("Max length: {}".format(max_length))

FileNotFoundError: ignored

In [ ]:
from datasets import load_dataset
data = load_dataset("json", data_files="alpaca_data.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-2affed7f197dfaba/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

NameError: ignored

In [ ]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""


alpacadata = data.map(lambda data_point: {"prompt": (generate_prompt(data_point))})

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
alpacadata['train']['prompt'][0]


'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Input:\n\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'

In [ ]:
data
train_dataset=data["train"]
train_dataset

Dataset({
    features: ['output', 'input', 'instruction', 'prompt'],
    num_rows: 52002
})

In [ ]:
len(descriptions[:1000])

1000

In [ ]:
tokenizer.eos_token, tokenizer.pad_token, tokenizer.bos_token,

NameError: ignored

In [ ]:
tokenizer??

In [ ]:
tokenizer.decode(dataset[0][0], skip_special_tokens=True)

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Input:\n\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'

In [ ]:
alpacaPrompts = alpacadata['train']['prompt'][:1000] ## only 1000 sets
# (alpacaPrompts[:1000][0])
max_length = max([len(tokenizer.encode(alpacaPrompt)) for alpacaPrompt in alpacaPrompts])
max_length

441

In [ ]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|endoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


dataset = NetflixDataset(alpacaPrompts, tokenizer, max_length=max_length)
train_size = int(0.2 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset)-train_size])


In [ ]:
(dataset[0])

In [ ]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=5, logging_steps=10,
                                  save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=6, per_device_eval_batch_size=4,
                                  warmup_ratio=0.1, weight_decay=0.01, logging_dir='./logs')

Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,7.265200
20,1.868900
30,0.529400
40,0.352300
50,0.360200
60,0.350300
70,0.368500
80,0.273900
90,0.272400
100,0.315000


TrainOutput(global_step=170, training_loss=0.8026257360682768, metrics={'train_runtime': 130.7007, 'train_samples_per_second': 7.651, 'train_steps_per_second': 1.301, 'total_flos': 224985019392000.0, 'train_loss': 0.8026257360682768, 'epoch': 5.0})

In [ ]:
model.generate??

In [ ]:
tokenizer??

In [ ]:
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.32.0.dev0"
}

In [ ]:
## sample prompt input
alpacaInput = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nMake a list of gift ideas.\n\n### Input:\n\n\n### Response:
"""

In [ ]:
model.generate??

In [ ]:
# generated = tokenizer(alpacaInput,return_tensors="pt").input_ids.cuda()
generated = tokenizer(alpacaInput,return_tensors="pt").input_ids.cuda()
generated_am = tokenizer(alpacaInput,return_tensors="pt").attention_mask.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                # bos_token='<|startoftext|>',
                                # eos_token='<|endoftext|>', pad_token='<|pad|>',
                                max_length=512, top_p=0.95, temperature=1.5, num_return_sequences=1)

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: 
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make a list of gift ideas.

### Input:


### Response:
Make the gift chosen in this task, along with any supplies, an ice cream and soda cube dispensing tip. Freeze all ingredients, then spoon over the ingredients you'd like in each scoop. Drizzle any desired amount over and include the ice cream into each scoop, or use any other ice cream ingredients.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("sandeshrajx/gpt-neo-125m-netflix", use_auth_token=True)

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sandeshrajx/gpt-neo-125m-netflix/commit/9c2cc15305cb2d56d0a9b5ecd9e6083341480fc5', commit_message='Upload GPTNeoForCausalLM', commit_description='', oid='9c2cc15305cb2d56d0a9b5ecd9e6083341480fc5', pr_url=None, pr_revision=None, pr_num=None)